# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [10]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć

import numpy as np

class ZnW:
    
    def __init__(self,X):
        self.n = 17
        self.W = [1,0,0,0,1]
        self.max_pow = len(self.W) - 1
        self.X = [num % self.n for num in self.polynomial_remainder(X)]


    def polynomial_remainder(self,dividend):
        divisor = self.W[:]
        
        while len(dividend) >= len(divisor):

            lead_coeff = dividend[0] / divisor[0]
            degree_diff = len(dividend) - len(divisor)
            

            scaled_divisor = [coeff * lead_coeff for coeff in divisor] + [0] * degree_diff
            dividend = [d - s for d, s in zip(dividend, scaled_divisor)]

            while dividend and dividend[0] == 0:
                dividend.pop(0)

            for i in range(len(dividend)):
                if dividend[i] != 0:
                    dividend[i] = int(dividend[i] % self.n)
        
        return dividend
    
    def __add__(self, o):
        len_self = len(self.X)
        len_other = len(o.X)

        if len_self < len_other:
            X_self = [0] * (len_other - len_self) + self.X
            X_other = o.X
        else:
            X_self = self.X
            X_other = [0] * (len_self - len_other) + o.X
        
        res = np.add(X_self,X_other)
        return ZnW(res)

    def __sub__(self, o):
        len_self = len(self.X)
        len_other = len(o.X)

        if len_self < len_other:
            X_self = [0] * (len_other - len_self) + self.X
            X_other = o.X
        else:
            X_self = self.X
            X_other = [0] * (len_self - len_other) + o.X

        res = [(x1 - x2) % self.n for x1, x2 in zip(X_self, X_other)]
        return ZnW(res)
        
    def __mul__(self,o):
        if isinstance(o,ZnW):
            len_self = len(self.X)
            len_other = len(o.X)

            if len_self < len_other:
                X_self = [0] * (len_other - len_self) + self.X
                X_other = o.X
            else:
                X_self = self.X
                X_other = [0] * (len_self - len_other) + o.X

            return ZnW(np.polymul(X_self,X_other))
        elif isinstance(o,int):
            return ZnW([i * o for i in self.X])
    
    def __rmul__(self, o):
        return self.__mul__(o)
    

    def __str__(self):
        return f"{" ".join(str(i) for i in self.X)}"

## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [2]:
def format_polynomial(znw_obj):
    coeffs = znw_obj.X
    terms = []
    for i, coeff in enumerate(reversed(coeffs)):
        if coeff != 0:
            term = f"{coeff}" if i == 0 else (f"{coeff}x^{i}" if i > 1 else f"{coeff}x")
            terms.append(term)
    return " + ".join(terms[::-1]) if terms else "0"

def print_t(t):
    print("\nWynik t:")
    formatted_t = [format_polynomial(poly) for poly in t]
    for i, poly in enumerate(formatted_t):
        print(f"t[{i}] = {poly}")

In [3]:
def is_zero_vector(X):
    for x in X:
        if x != 0:
            return False
    return True

In [4]:
import secrets
import random
def key_gen(test=False):
    A = []
    s = []
    e = []
    t = [ZnW([0]), ZnW([0])]
    if test:
        A = [[ZnW([secrets.randbelow(17) for _ in range(4)]) for _ in range(2)] for _ in range(2)]
        while True:
            s_temp = [ZnW([secrets.choice([-1,0,0,0,0,0,0,0,0,1]) for _ in range(4)]) for _ in range(2)]
            if not all(is_zero_vector(vec.X) for vec in s_temp):
                s = s_temp
                break
        
        while True:
            e_temp = [ZnW([secrets.choice([-1,0,0,0,0,0,0,0,0,1]) for _ in range(4)]) for _ in range(2)]
            if not all(is_zero_vector(vec.X) for vec in e_temp):
                e = e_temp
                break
    else:
        A = [[ZnW([6, 16, 16, 11]), ZnW([9, 4, 6, 3])],[ZnW([5, 3, 10, 1]), ZnW([6, 1, 9, 15])]]
        s = [ZnW([-1, -1, 1, 0]), ZnW([-1, 0, -1, 0])]
        e = [ZnW([0, 1, 0, 0]), ZnW([0, 1, -1, 0])]

    for i in range(2):
        for j in range(2):
            t[i] = t[i] + A[i][j] * s[j]
        t[i] = t[i] + e[i]

    return A, t, s

A, t, s = key_gen()
print_t(t)


Wynik t:
t[0] = 16x^3 + 15x^2 + 7
t[1] = 10x^3 + 12x^2 + 11x + 6


### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [5]:
def print_encryption_result(u, v):
    print("\nZaszyfrowane wartości:\n")

    print("u = [")
    for i, poly in enumerate(u):
        formatted_poly = format_polynomial(poly)
        print(f"  u[{i}] = {formatted_poly}")
    print("]")

    print("\nv = ", format_polynomial(v))

In [6]:
def encrypt(A, t, m, test=False):
    q = 17
    q_div = q // 2 + 1
    if test:
        while True:
            r_temp = [ZnW([secrets.choice([-1,0,0,0,0,0,0,0,0,1]) for _ in range(4)]) for _ in range(2)]
            if not all(is_zero_vector(vec.X) for vec in r_temp):
                r = r_temp
                break
        else:
            if all(is_zero_vector(vec.X) for vec in r):
                raise ValueError("r cannot be all zero vectors")
    
        while True:
            e1_temp = [ZnW([secrets.choice([-1,0,0,0,0,0,0,0,0,1]) for _ in range(4)]) for _ in range(2)]
            if not all(is_zero_vector(vec.X) for vec in e1_temp):
                e1 = e1_temp
                break
        else:
            if all(is_zero_vector(vec.X) for vec in e1):
                raise ValueError("e1 cannot be all zero vectors")
        
        while True:
            e2_temp = ZnW([secrets.choice([-1,0,0,0,0,0,0,0,0,1]) for _ in range(4)])
            if not is_zero_vector(e2_temp.X):
                e2 = e2_temp
                break
    else:
        r = [ZnW([-1, 1, 0, 0]), ZnW([1, 1, 0, -1])]
        e1 = [ZnW([0, 1, 1, 0]), ZnW([0, 1, 0, 0])]
        e2 = ZnW([-1, -1, 0, 0])
    m_poly = ZnW(m)

    u = [ZnW([0]), ZnW([0])]
    for i in range(2):
        for j in range(2):
            u[i] = u[i] + A[j][i] * r[j]
        u[i] = u[i] + e1[i]

    v = ZnW([0])
    for i in range(2):
        v = v + t[i] * r[i]
    v = v + e2 + (m_poly * q_div)
    return u, v
    
A, t, s = key_gen()
m = [1, 0, 1, 1]
u, v = encrypt(A, t, m)

print_encryption_result(u, v)


Zaszyfrowane wartości:

u = [
  u[0] = 11.0x^3 + 11.0x^2 + 10.0x + 3.0
  u[1] = 4x^3 + 4x^2 + 13x + 11
]

v =  8x^3 + 6x^2 + 9x + 16


### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [7]:
def print_decryption_result(m):
    print("\nZdeszyfrowane wartości:\n")

    formatted_m = "".join(str(bit) for bit in m)
    print(f"m = {formatted_m}")

In [8]:
def decrypt(c, s, test=False):
    u, v = c

    sTu = ZnW([0])
    for i in range(len(s)):
        sTu = sTu + s[i] * u[i]

    m_n = v - sTu
    m_binary = [
        1 if coeff >= 5 and coeff <= 12 else 0
        for coeff in m_n.X
    ]

    return m_binary
    
c = u, v
m = decrypt(c, s)
print_decryption_result(m)


Zdeszyfrowane wartości:

m = 1011


### Testy

In [16]:
import secrets as sc
import random

for k in range(10):
    success = 0
    for i in range(1000):
        output = []
        A,t,s = key_gen(test=True)
        
        m=[sc.choice((0,1)) for k in range(4)]
        
        c = encrypt(A,t,m, test=True)
        m_n = decrypt(c,s, test=True)
    
        if m_n == m:
            success += 1
    
    print(f'Success rate: {success * 100 /1000} %')


Success rate: 99.6 %
Success rate: 99.8 %
Success rate: 99.9 %
Success rate: 99.7 %
Success rate: 99.8 %
Success rate: 99.8 %
Success rate: 99.8 %
Success rate: 99.3 %
Success rate: 99.7 %
Success rate: 99.9 %
